<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Salvando_e_carregando_o_modelo_(ex7)_e_classificando_somente_um_registro_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando as bibliotecas

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
torch.__version__

'2.5.1+cu124'

## Base de dados

In [2]:
np.random.seed(123)
torch.manual_seed(123)

In [11]:
base = pd.read_csv('/content/iris.csv')
previsores = base.iloc[:, 0:4].values #selecionando todas as linhas e as 3 primeiras colunas
classe = base.iloc[:, 4].values #selecionando todas as linhas e apenas a 4 coluna

In [12]:
# convertendo valores categóricos em numéricos (ex: flor -> 1, petala -> 2)
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [13]:
# transformando de np array para tensores:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [17]:
previsores.shape

torch.Size([150, 4])

In [18]:
classe.shape

torch.Size([150])

## Criando um dataset com previsores e classe

In [19]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), # criando um dataset com o TensorDataset agrupando as entradas(previsores) e os rótulos (classe)
                                           batch_size = 10, # cada batch conterá 10 amostras
                                           shuffle = True) # os dados serão embaralhados a cada épooca, garantindo que o modelo não decore a ordem dos dados


## Construção do modelo

In [21]:
class ClassificadorTorch (nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(4,16)
    torch.nn.init.uniform_(self.dense0.weight, -0.05, 0.05)
    self.dense1 = nn.Linear(16,16)
    torch.nn.init.uniform_(self.dense1.weight, -0.05, 0.05)
    self.dense2 = nn.Linear(16,16)
    torch.nn.init.uniform_(self.dense2.weight, -0.05, 0.05)
    self.dense3 = nn.Linear(16,3)

    self.activation = nn.ReLU()
    self.dropout = nn.Dropout(0.2)
    self.output = nn.Softmax(dim=1)

  def forward(self,X):
    X = self.dense0(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense1(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense2(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense3(X)
    X = self.output(X)
    return X

In [22]:
classificador = ClassificadorTorch()

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classificador.parameters(),
                             lr = 0.001,
                             weight_decay = 0.0001)

## Treinando o modelo

In [25]:
for epoch in range (500):
  running_loss = 0.

  for data in train_loader:
    inputs, labels = data
    optimizer.zero_grad()

    outputs = classificador(inputs)
    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'Época {epoch+1:3d}: perda {running_loss/len(train_loader):5f}')

Época   1: perda 1.099640
Época   2: perda 1.098976
Época   3: perda 1.098895
Época   4: perda 1.098685
Época   5: perda 1.098947
Época   6: perda 1.098454
Época   7: perda 1.097411
Época   8: perda 1.094959
Época   9: perda 1.087593
Época  10: perda 1.071121
Época  11: perda 1.046149
Época  12: perda 1.020933
Época  13: perda 1.003415
Época  14: perda 0.969156
Época  15: perda 0.929676
Época  16: perda 0.903507
Época  17: perda 0.873580
Época  18: perda 0.850458
Época  19: perda 0.847349
Época  20: perda 0.836337
Época  21: perda 0.812754
Época  22: perda 0.810298
Época  23: perda 0.810559
Época  24: perda 0.802541
Época  25: perda 0.805113
Época  26: perda 0.769174
Época  27: perda 0.767507
Época  28: perda 0.750703
Época  29: perda 0.756477
Época  30: perda 0.739609
Época  31: perda 0.745320
Época  32: perda 0.710760
Época  33: perda 0.731333
Época  34: perda 0.723003
Época  35: perda 0.705042
Época  36: perda 0.688156
Época  37: perda 0.693372
Época  38: perda 0.704842
Época  39: p

## Classificando somente um registro

In [40]:
novo_registro = torch.rand((1,4)) # criando um unico registro com 4 features aleatorias

In [41]:
classificador.eval() # colocando o modelo em modo avaliacao

ClassificadorTorch(
  (dense0): Linear(in_features=4, out_features=16, bias=True)
  (dense1): Linear(in_features=16, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=16, bias=True)
  (dense3): Linear(in_features=16, out_features=3, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Softmax(dim=1)
)

In [42]:
with torch.no_grad(): # evita fazer calculos desnecessarios de gradiente
  saida = classificador(novo_registro)

In [43]:
classe_predita = torch.argmax(saida, dim=1).item() # obtendo a classe com maior probabilidade

In [44]:
# exibindo resultado da predição:
print(f'Valores de entrada: {novo_registro}')
print(f'valores de saída do modelo: {saida}')
print(f'classe prevista pelo modelo: {classe_predita}')

Valores de entrada: tensor([[0.5505, 0.5295, 0.2579, 0.2716]])
valores de saída do modelo: tensor([[9.3322e-01, 6.6784e-02, 1.0100e-16]])
classe prevista pelo modelo: 0


## Salvando o classificador

In [45]:
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-2.9181e-02, -1.3845e-01,  2.5689e-01,  4.1150e-01],
                      [-8.3636e-04, -1.2133e-01,  2.4632e-01,  3.4094e-01],
                      [ 1.8702e-01,  2.4925e-01, -2.7884e-01, -3.8794e-01],
                      [ 1.5770e-01,  2.8071e-01, -2.9096e-01, -4.2115e-01],
                      [-5.2147e-02, -1.7987e-01,  2.7219e-01,  3.8150e-01],
                      [ 1.6671e-01,  1.8002e-01, -2.6353e-01, -4.4664e-01],
                      [-6.6180e-02, -2.2173e-01,  3.1148e-01,  5.0832e-01],
                      [-3.6549e-02, -2.1154e-01,  3.5635e-01,  4.5821e-01],
                      [-8.9790e-03, -2.6542e-03, -5.8782e-03, -4.6132e-04],
                      [-5.1015e-21,  4.2077e-40,  1.9431e-27, -4.3233e-39],
                      [ 1.0702e-01,  2.2001e-01, -2.0757e-01, -4.2032e-01],
                      [-1.5152e-02, -2.2686e-01,  3.0582e-01,  4.5120e-01],
                      [ 1.3989e-01,  2.2850e-01, -2.5489e

In [46]:
# salvando o classificador com o nome 'classificador_iris' na extensao .pth
torch.save(classificador.state_dict(), 'classificador_iris.pth')